# Learn rotation params from |0> to |1>

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Pure qisqit

In [1]:
from qiskit import QuantumCircuit, Aer, execute, aqua
from qiskit.circuit import Parameter

In [35]:
def my_exp(counts):
    if '0' not in counts:
        counts['0'] = 0
        
    if '1' not in counts:
        counts['1'] = 0
        
    return counts['1'] / (counts['0'] + counts['1'])

In [65]:
params_names = [Parameter('rx'), Parameter('ry')]
backend = Aer.get_backend('qasm_simulator')

qc = QuantumCircuit(1)
qc.rx(params_names[0], 0)
qc.ry(params_names[1], 0)

qc.measure_all()



In [72]:
def execute_qc(params):
    binds = [{param:val for param, val in zip(params_names, params)}]
    counts = execute(qc, backend, parameter_binds=binds).result().get_counts()
    return my_exp(counts)

In [73]:
execute_qc([1.5, 0.2])

0.4638671875

In [86]:
def loss(params):
    return 1 - execute_qc(params)

In [87]:
def my_grad(loss, params):
    grad = [0] * len(params)
    for k in range(len(params)):
        curr_params = params.copy()
        curr_params[k] = params[k] + 3.14 / 2
        up = loss(curr_params)
        curr_params[k] = params[k] - 3.14 / 2
        down = loss(curr_params)
        
        grad[k] = (up - down) / 2
        
    return grad

In [92]:
weights = np.random.uniform(0, np.pi, 2)
for _ in range(20):
    loss_val = loss(weights)
    print(loss_val)
    grad_val = my_grad(loss, weights)
    grad_val = np.array(grad_val)

    weights = weights - 0.3 * grad_val

0.517578125
0.5009765625
0.4814453125
0.447265625
0.4677734375
0.4560546875
0.4501953125
0.3984375
0.3857421875
0.3212890625
0.326171875
0.294921875
0.259765625
0.2119140625
0.189453125
0.166015625
0.1259765625
0.0966796875
0.0712890625
0.060546875


## Pennylane

In [95]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

In [96]:
dev = qml.device("default.qubit", wires=1)
#dev = qml.device("qiskit.aer", wires=1)

@qml.qnode(dev, diff_method="parameter-shift", mutable=False)
def circuit(weights):
    qml.RX(weights[0], wires=0)
    qml.RY(weights[1], wires=0)
    return qml.expval(qml.PauliZ(0))

def loss(weight):
    return circuit(weight)

In [97]:
weights = np.random.uniform(0, np.pi, 2)
for _ in range(20):
    loss_val = loss(weights)
    print(loss_val)
    grad_fn = qml.grad(loss, argnum=0)
    grad_val = grad_fn(weights)
    grad_val = np.array(grad_val)

    weights = weights - 0.3 * grad_val

0.6406698714224601
0.4484584288159242
0.19565173346260845
-0.09709517459234368
-0.38554315084284313
-0.6230018795488429
-0.7877958300334716
-0.8875472591279022
-0.9425572389928326
-0.9712476603063374
-0.9857603847936782
-0.9929856469564871
-0.996554013649663
-0.99830930702096
-0.9991710407246023
-0.9995936850321429
-0.9998008756553608
-0.9999024218637016
-0.999952184982484
-0.999976570225842


In [4]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
weights = np.random.uniform(0, np.pi, 2)
for i in range(20):
    loss_val = loss(weights)
    print(loss_val)
    weights = opt.step(loss, weights)
    

-0.08780622982958786
-0.20746865945368925
-0.3659172714182596
-0.5604852852244977
-0.7485228438416808
-0.8814327247542308
-0.9514485078173389
-0.9815683127074191
-0.9932290776060018
-0.9975442762985759
-0.9991135511529649
-0.9996805674345894
-0.9998849639050983
-0.9999585817705153
-0.999985088758739
-0.9999946318651864
-0.9999980674600671
-0.9999993042841467
-0.9999997495421011
-0.9999999098351315
